In [1]:
import numpy as np
import pandas as pd


from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Load Data

In [2]:
iris = load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

X = df.iloc[:, 0:4]
y = df.iloc[:, -1]

## Train Test Split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

## Scale the data

In [95]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Dataloader

In [96]:
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [97]:
train_data = trainData(torch.FloatTensor(X_train), torch.LongTensor(y_train.values))
test_data = testData(torch.FloatTensor(X_test))

## Define NN model

In [142]:
class PreTrainedModel(nn.Module):
    def __init__(self, num_feature, num_class):
        super(PreTrainedModel, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 6)
        self.layer_2 = nn.Linear(6, 8)
        self.layer_out = nn.Linear(8, num_class)
        
        self.relu = nn.ReLU()
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        
        return x

In [143]:
NUM_INPUT_FEATURES = len(X.columns)
NUM_OUTPUT_CLASSES = 3
EPOCHS = 8
BATCH_SIZE = 4

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=True)

In [144]:
model = PreTrainedModel(num_feature = NUM_INPUT_FEATURES, num_class=NUM_OUTPUT_CLASSES)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

PreTrainedModel(
  (layer_1): Linear(in_features=4, out_features=6, bias=True)
  (layer_2): Linear(in_features=6, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=3, bias=True)
  (relu): ReLU()
)


## Train Model

In [145]:
model.train()
for e in range(EPOCHS):
    epoch_loss = 0
    epoch_acc = 0
    
    for X_batch, y_batch in train_loader:
        y_pred_probs = model(X_batch)
        loss = criterion(y_pred_probs, y_batch)
        
        loss.backward()
        optimizer.step()
        
        _, y_pred = torch.max(y_pred_probs, dim = 1)
        batch_acc = (y_pred == y_batch).sum().item()/BATCH_SIZE
        
        epoch_loss += loss.item()
        epoch_acc += batch_acc
    
    print(f'Epoch {e+0:02}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.5f}')    

Epoch 00: | Loss: 1.13740 | Acc: 0.34167
Epoch 01: | Loss: 1.10503 | Acc: 0.45000
Epoch 02: | Loss: 1.06221 | Acc: 0.33333
Epoch 03: | Loss: 0.98183 | Acc: 0.40833
Epoch 04: | Loss: 0.88134 | Acc: 0.58333
Epoch 05: | Loss: 0.77200 | Acc: 0.65000
Epoch 06: | Loss: 0.68214 | Acc: 0.74167
Epoch 07: | Loss: 0.60828 | Acc: 0.83333


## Test Model

In [146]:
y_pred_list = []

with torch.no_grad():
    for X_test_batch in test_loader:
        y_pred_probs = model(X_test_batch)
        _, y_pred = torch.max(y_pred_probs, dim = 1)
        
        y_pred_list.append(y_pred.numpy())

In [147]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.20      0.20      0.20        10
         1.0       0.20      0.25      0.22         8
         2.0       0.40      0.33      0.36        12

    accuracy                           0.27        30
   macro avg       0.27      0.26      0.26        30
weighted avg       0.28      0.27      0.27        30



## Weight Pruning

In [158]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, ": ",  param.shape, "\n", param.data, "\n")
        print("=" * 100)

layer_1.weight :  torch.Size([6, 4]) 
 tensor([[-0.2648,  0.1227, -0.7883, -0.0803],
        [-0.1095, -0.6869,  0.0119, -0.5226],
        [-0.1791,  0.2791, -0.5246, -0.6669],
        [-0.2769, -0.1220,  0.2103, -0.1768],
        [ 0.4458,  0.5001,  0.2323,  0.6193],
        [-0.4455,  0.7174, -0.3018, -0.7256]]) 

layer_1.bias :  torch.Size([6]) 
 tensor([ 0.3653,  0.2289,  0.4179, -0.4063, -0.2165,  0.1836]) 

layer_2.weight :  torch.Size([8, 6]) 
 tensor([[ 0.5686, -0.1121,  0.5220, -0.1004, -0.4658,  0.5302],
        [ 0.3992, -0.2845,  0.7956, -0.2060, -0.0964,  0.5952],
        [-0.4409, -0.6192, -0.3328, -0.1938, -0.4393, -0.1782],
        [ 0.4532,  0.5382, -0.5189, -0.2133, -0.1340,  0.0403],
        [ 0.0009, -0.3969,  0.1155,  0.0045,  0.0661, -0.2889],
        [-0.1400,  0.0989, -0.1770, -0.0407,  0.4508, -0.0686],
        [-0.0869, -0.2890, -0.1779,  0.3563, -0.1222, -0.2564],
        [-0.1229, -0.2406,  0.0193, -0.0092, -0.3095,  0.0606]]) 

layer_2.bias :  torch.Size([8